# Summary

Test out new batch query functionality.

In [57]:
%load_ext autoreload
%autoreload 2

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
import matplotlib.pyplot as plt
import numpy as np
import os
import openai
import pandas as pd
from pathlib import Path

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    GPTBackend, query_kwargs_grid, MOCKS, postprocess_gpt_response, \
    containerize, truncate_at_first_stop, query_gpt_mock
from jabberwocky.utils import strip
from htools import *

In [59]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [60]:
gpt = GPTBackend()
gpt.switch('repeat')

Switching openai backend to "repeat".


In [72]:
def postprocess_response(response, n, trunc_full=True, strip_output=True, 
                         **kwargs):
    text, full_response = containerize(*response)
    print('TEXT:', text)
    print('FULL:', full_response)

    # Manually check for stop phrases because most backends either don't
    # or truncate AFTER the stop phrase which is rarely what we want.
    stop = kwargs.get('stop', [])
    clean_text = []
    clean_full = []
    for i, (text_, resp_) in enumerate(zip(text, full_response)):
        text_ = truncate_at_first_stop(
            text_,
            stop_phrases=stop,
            finish_reason=resp_.get('finish_reason', ''),
            trunc_full=trunc_full,
            trunc_partial=True
        )
        clean_text.append(strip(text_, strip_output))
        clean_full.append({**resp_, 'prompt_index': i // n})

    return clean_text, clean_full

In [67]:
def test_backend(backend, results=None):
    def gen():
        yield from results.kwargs
        
    with gpt(backend):
        kwargs_list = []
        resp_list = []
        gen_kwargs = gen if results else query_kwargs_grid
        for i, kwargs in enumerate(gen_kwargs(), start=1):
            print(f'\n\n{i}.')
            if results:
                res = results.res[i - 1]
            else:
                res = gpt.query(**kwargs)
            if kwargs['stream']:
                cur = []
                for tok, tok_full in res:
                    cur.append((tok, tok_full))
                    print(tok)
                    print('\t' + str(tok_full) + '\n')
                    if tok_full['finish_reason']: print('\n---\n')
            else:
                texts, fulls = res
                print('TEXTS:', texts)
                print('FULLS:', fulls)
                cur = res
            print(spacer())

            kwargs_list.append(kwargs)
            resp_list.append(cur)
    return Results(kwargs=kwargs_list, res=resp_list)

Issues

- no prompt_index in stream=True mode for either repeat or banana (pretty sure not for paid backends either)

In [118]:
repeat_res = test_backend('repeat')

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.
Switching openai backend to "repeat".
np>1: False
nc>1: False
stream: False


1.
{'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
TEXTS: ['YESTERDAY WAS']
FULLS: [{'prompt_index': 0}]

-------------------------------------------------------------------------------

np>1: False
nc>1: False
stream: True


2.
{'n': 1, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
YESTERDAY 
	{'index': 0, 'prompt_index': 0, 'finish_reason': None}

WAS 
	{'index': 0, 'prompt_index': 0, 'finish_reason': 'dummy'}


---


------------------

/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:372: UserWarning: Unused kwargs {'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3} received by query_gpt_repeat.
  warnings.warn(f'Unused kwargs {kwargs} received by query_gpt_repeat.')
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:669: UserWarning: strip_output=True is not supported in stream mode. Automatically setting it to False.
  warnings.warn('strip_output=True is not supported in stream '
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:674: UserWarning: Streaming mode does not support manual truncation of stop phrases and your current backend has limited support for truncation.
  'Streaming mode does not support manual truncation of '
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:372: UserWarning: Unused kwargs {'n': 1, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3} received by query_gpt_repeat.
  warnings.warn(f'Unused kwargs {kwargs} received by qu

In [29]:
# Even though it's free, try to avoid hitting the API too much just in case 
# they have some rate limit.
banana_res = test_backend('banana')
save(banana_res, 'data/tmp/banana_res.pkl')
banana_res = load('data/tmp/banana_res.pkl')

Switching openai backend to "banana".
np>1: True
nc>1: True
stream: True
{'n': 2, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'banana', 'query_func': 'query_gpt_banana'}}
{'n': 2, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'How many', 'meta': {'backend_name': 'banana', 'query_func': 'query_gpt_banana'}}


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:421: UserWarning: query_gpt_banana received unused kwargs {'stream': True, 'engine_i': 0, 'logprobs': 3}.
  warnings.warn(f'query_gpt_banana received unused kwargs {kwargs}.')




1.
 
	{'id': '563b1b8f-eb21-4582-8cf3-715a0e01cb74', 'message': 'success', 'created': 1649818659, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a big day', 'input': 'Yesterday was'}], 'index': 0, 'prompt_index': 0, 'finish_reason': None}

a 
	{'id': '563b1b8f-eb21-4582-8cf3-715a0e01cb74', 'message': 'success', 'created': 1649818659, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a big day', 'input': 'Yesterday was'}], 'index': 0, 'prompt_index': 0, 'finish_reason': None}

big 
	{'id': '563b1b8f-eb21-4582-8cf3-715a0e01cb74', 'message': 'success', 'created': 1649818659, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a big day', 'input': 'Yesterday was'}], 'index': 0, 'prompt_index': 0, 'finish_reason': None}

day 
	{'id': '563b1b8f-eb21-4582-8cf3-715a0e01cb74', 'message': 'success', 'created': 1649818659, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a big day', 'input': 'Yesterday was'}], 'index': 0, 'prompt_index': 0, 'finish_reason':

/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:421: UserWarning: query_gpt_banana received unused kwargs {'stream': False, 'engine_i': 0, 'logprobs': 3}.
  warnings.warn(f'query_gpt_banana received unused kwargs {kwargs}.')




2.
TEXTS: ['a great day', 'my first day', 'of you have', 'more weeks are']
FULLS: [{'id': '0626ac30-7312-4102-a60c-f0b43bf91c7c', 'message': 'success', 'created': 1649818660, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a great day', 'input': 'Yesterday was'}], 'prompt_index': 0}, {'id': '83c796e2-157e-40b0-86c6-e12cf922f5a9', 'message': 'success', 'created': 1649818660, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' my first day', 'input': 'Yesterday was'}], 'prompt_index': 0}, {'id': '71ac8cfc-c798-4ca5-838a-97c16588fbd3', 'message': 'success', 'created': 1649818660, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' of you have', 'input': 'How many'}], 'prompt_index': 1}, {'id': '2c16ad55-1958-43de-a79f-19ef01a0840e', 'message': 'success', 'created': 1649818660, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' more weeks are', 'input': 'How many'}], 'prompt_index': 1}]

--------------------------------------------------------------------

/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:421: UserWarning: query_gpt_banana received unused kwargs {'n': 1, 'stream': True, 'engine_i': 0, 'logprobs': 3}.
  warnings.warn(f'query_gpt_banana received unused kwargs {kwargs}.')




3.
 
	{'id': '87438ad7-c243-4cc1-8660-45c15f1bc1fe', 'message': 'success', 'created': 1649818661, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' the first day', 'input': 'Yesterday was'}], 'index': 0, 'prompt_index': 0, 'finish_reason': None}

the 
	{'id': '87438ad7-c243-4cc1-8660-45c15f1bc1fe', 'message': 'success', 'created': 1649818661, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' the first day', 'input': 'Yesterday was'}], 'index': 0, 'prompt_index': 0, 'finish_reason': None}

first 
	{'id': '87438ad7-c243-4cc1-8660-45c15f1bc1fe', 'message': 'success', 'created': 1649818661, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' the first day', 'input': 'Yesterday was'}], 'index': 0, 'prompt_index': 0, 'finish_reason': None}

day 
	{'id': '87438ad7-c243-4cc1-8660-45c15f1bc1fe', 'message': 'success', 'created': 1649818661, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' the first day', 'input': 'Yesterday was'}], 'index': 0, 'prompt_index':

/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:421: UserWarning: query_gpt_banana received unused kwargs {'n': 1, 'stream': False, 'engine_i': 0, 'logprobs': 3}.
  warnings.warn(f'query_gpt_banana received unused kwargs {kwargs}.')




4.
TEXTS: ['the first day', 'minutes are there']
FULLS: [{'id': '4e38743f-be5d-4ee1-b9b4-bd3478a57826', 'message': 'success', 'created': 1649818661, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' the first day', 'input': 'Yesterday was'}], 'prompt_index': 0}, {'id': 'e67d74e2-3c49-44d0-9103-3f29354049f8', 'message': 'success', 'created': 1649818661, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' minutes are there', 'input': 'How many'}], 'prompt_index': 1}]

-------------------------------------------------------------------------------

np>1: False
nc>1: True
stream: True
{'n': 2, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'banana', 'query_func': 'query_gpt_banana'}}


5.
 
	{'id': '07a2463a-aa72-43ec-bccd-740afc5c0142', 'message': 'success', 'created': 1649818662, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' the first day', 'input': 'Yesterday was'}], 'index': 0, 'prompt_in

In [28]:
banana_res = test_backend('banana', results=banana_res)

Switching openai backend to "banana".


1.
 
	{'id': 'ab2f3b0a-0011-495d-848a-62bc558484a1', 'message': 'success', 'created': 1649736961, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a good day', 'input': 'Yesterday was'}], 'index': 0, 'finish_reason': None}

a 
	{'id': 'ab2f3b0a-0011-495d-848a-62bc558484a1', 'message': 'success', 'created': 1649736961, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a good day', 'input': 'Yesterday was'}], 'index': 0, 'finish_reason': None}

good 
	{'id': 'ab2f3b0a-0011-495d-848a-62bc558484a1', 'message': 'success', 'created': 1649736961, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a good day', 'input': 'Yesterday was'}], 'index': 0, 'finish_reason': None}

day 
	{'id': 'ab2f3b0a-0011-495d-848a-62bc558484a1', 'message': 'success', 'created': 1649736961, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a good day', 'input': 'Yesterday was'}], 'index': 0, 'finish_reason': 'dummy'}


---

 
	{'id': '896ed

In [119]:
mock_res = test_backend('mock')

Switching openai backend to "mock".
np>1: False
nc>1: False
stream: False


1.
{'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'mock', 'query_func': 'query_gpt_mock'}}
TEXTS: ['a typical rainy']
FULLS: [{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x125cce570> JSON: {
  "text_offset": [
    0,
    2,
    10
  ],
  "token_logprobs": [
    -1.3818359375,
    -6.06640625,
    -5.671875
  ],
  "tokens": [
    " a",
    " typical",
    " rainy"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    },
    {
      " big": -1.83984375,
      " day": -2.53125,
      " very": -2.916015625
    },
    {
      " Monday": -2.974609375,
      " day": -1.095703125,
      " work": -2.94140625
    }
  ]
}, 'text': ' a typical rainy', 'token_index': 0, 'prompt_index': 0}]

-----------------------------------------------------------------------

{'n': 1, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'How many', 'meta': {'backend_name': 'mock', 'query_func': 'query_gpt_mock'}}
 a
	{'finish_reason': None, 'index': 0, 'logprobs': <OpenAIObject at 0x125cce048> JSON: {
  "text_offset": [
    0
  ],
  "token_logprobs": [
    -1.3818359375
  ],
  "tokens": [
    " a"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    }
  ]
}, 'text': ' a', 'token_index': 0}

 bit
	{'finish_reason': None, 'index': 0, 'logprobs': <OpenAIObject at 0x125cce200> JSON: {
  "text_offset": [
    2
  ],
  "token_logprobs": [
    -4.75390625
  ],
  "tokens": [
    " bit"
  ],
  "top_logprobs": [
    {
      " big": -1.83984375,
      " day": -2.53125,
      " very": -2.916015625
    }
  ]
}, 'text': ' bit', 'token_index': 1}

 busy
	{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x125cce3b8> JSON: {
  "text_offset": [
    6
  ],
  "token_logprobs"

{'n': 2, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'How many', 'meta': {'backend_name': 'mock', 'query_func': 'query_gpt_mock'}}
 a
	{'finish_reason': None, 'index': 0, 'logprobs': <OpenAIObject at 0x125cec1a8> JSON: {
  "text_offset": [
    0
  ],
  "token_logprobs": [
    -1.3818359375
  ],
  "tokens": [
    " a"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    }
  ]
}, 'text': ' a', 'token_index': 0}

 positive
	{'finish_reason': None, 'index': 0, 'logprobs': <OpenAIObject at 0x125cec360> JSON: {
  "text_offset": [
    2
  ],
  "token_logprobs": [
    -8.34375
  ],
  "tokens": [
    " positive"
  ],
  "top_logprobs": [
    {
      " big": -1.83984375,
      " day": -2.53125,
      " very": -2.916015625
    }
  ]
}, 'text': ' positive', 'token_index': 1}

 day
	{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x125cec518> JSON: {
  "text_offset": [
    11
  ],
  "tok

/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:400: UserWarning: query_gpt_mock received unused kwargs: {'engine_i': 0, 'max_tokens': 3, 'logprobs': 3}
  warnings.warn(f'query_gpt_mock received unused kwargs: {kwargs}')


In [109]:
for k, v in MOCKS.items():
    if k[-1]: continue
    print(k)
    print([row['text'] for row in v['choices']])

(True, True, False)
[" Mozilla's ann", ' reminded of this', ' lessons would it', ' teenagers are allowed']
(True, False, False)
[' 24 years ago', ' months did it']
(False, True, False)
[' my mother�', ' the Academy Awards']
(False, False, False)
[' a typical rainy']


## Scratch

In [51]:
# 2 prompts, 1 completion per prompt, stream=False
p2c1s0 = MOCKS[True, False, False]
p1c2s0 = MOCKS[False, True, False]

In [52]:
texts, fulls = containerize(*postprocess_gpt_response(p2c1s0))
postprocess_response((texts, fulls), n=1)

(['24 years ago', 'months did it'],
 [{'finish_reason': 'length',
   'index': 0,
   'logprobs': <OpenAIObject at 0x125bacaf0> JSON: {
     "text_offset": [
       0,
       3,
       9
     ],
     "token_logprobs": [
       -8.1484375,
       -1.931640625,
       -0.8271484375
     ],
     "tokens": [
       " 24",
       " years",
       " ago"
     ],
     "top_logprobs": [
       {
         " a": -1.3818359375,
         " my": -2.384765625,
         " the": -1.8720703125
       },
       {
         " hours": -1.21484375,
         " years": -1.931640625,
         "th": -1.8212890625
       },
       {
         " ago": -0.8271484375,
         " since": -1.0576171875,
         " to": -2.794921875
       }
     ]
   },
   'text': ' 24 years ago',
   'token_index': 0,
   'prompt_index': 0},
  {'finish_reason': 'length',
   'index': 1,
   'logprobs': <OpenAIObject at 0x125bacca8> JSON: {
     "text_offset": [
       0,
       7,
       11
     ],
     "token_logprobs": [
       -5.464843

In [73]:
texts, fulls = postprocess_gpt_response(p1c2s0)
postprocess_response((texts, fulls), n=2)

TEXT: [' my mother�', ' the Academy Awards']
FULL: [{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x1245f1990> JSON: {
  "text_offset": [
    0,
    3,
    10
  ],
  "token_logprobs": [
    -2.384765625,
    -5.4375,
    -0.44677734375
  ],
  "tokens": [
    " my",
    " mother",
    "\ufffd"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    },
    {
      " birthday": -2.107421875,
      " first": -1.3818359375,
      " last": -2.130859375
    },
    {
      "'s": -1.509765625,
      "-": -2.630859375,
      "\ufffd": -0.44677734375
    }
  ]
}, 'text': ' my mother�', 'token_index': 0}, {'finish_reason': 'length', 'index': 1, 'logprobs': <OpenAIObject at 0x1245f1b48> JSON: {
  "text_offset": [
    0,
    4,
    12
  ],
  "token_logprobs": [
    -1.8720703125,
    -8.796875,
    -0.1270751953125
  ],
  "tokens": [
    " the",
    " Academy",
    " Awards"
  ],
  "top_logprobs": [
    {
      " a":

(['my mother�', 'the Academy Awards'],
 [{'finish_reason': 'length',
   'index': 0,
   'logprobs': <OpenAIObject at 0x1245f1990> JSON: {
     "text_offset": [
       0,
       3,
       10
     ],
     "token_logprobs": [
       -2.384765625,
       -5.4375,
       -0.44677734375
     ],
     "tokens": [
       " my",
       " mother",
       "\ufffd"
     ],
     "top_logprobs": [
       {
         " a": -1.3818359375,
         " my": -2.384765625,
         " the": -1.8720703125
       },
       {
         " birthday": -2.107421875,
         " first": -1.3818359375,
         " last": -2.130859375
       },
       {
         "'s": -1.509765625,
         "-": -2.630859375,
         "\ufffd": -0.44677734375
       }
     ]
   },
   'text': ' my mother�',
   'token_index': 0,
   'prompt_index': 0},
  {'finish_reason': 'length',
   'index': 1,
   'logprobs': <OpenAIObject at 0x1245f1b48> JSON: {
     "text_offset": [
       0,
       4,
       12
     ],
     "token_logprobs": [
       -1.

In [74]:
texts

[' my mother�', ' the Academy Awards']

In [115]:
with gpt('mock'):
    res = gpt.query('ac', n=1, stream=False)

Switching openai backend to "mock".
{'n': 1, 'stream': False, 'prompt': 'ac', 'meta': {'backend_name': 'mock', 'query_func': 'query_gpt_mock'}}
>>> response: ([' a typical rainy'], [{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x125a06e08> JSON: {
  "text_offset": [
    0,
    2,
    10
  ],
  "token_logprobs": [
    -1.3818359375,
    -6.06640625,
    -5.671875
  ],
  "tokens": [
    " a",
    " typical",
    " rainy"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    },
    {
      " big": -1.83984375,
      " day": -2.53125,
      " very": -2.916015625
    },
    {
      " Monday": -2.974609375,
      " day": -1.095703125,
      " work": -2.94140625
    }
  ]
}, 'text': ' a typical rainy', 'token_index': 0}])
TEXT: [' a typical rainy']
FULL: [{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x125a06e08> JSON: {
  "text_offset": [
    0,
    2,
    10
  ],
  "token_logprobs":

In [116]:
res

(['a typical rainy'],
 [{'finish_reason': 'length',
   'index': 0,
   'logprobs': <OpenAIObject at 0x125a06e08> JSON: {
     "text_offset": [
       0,
       2,
       10
     ],
     "token_logprobs": [
       -1.3818359375,
       -6.06640625,
       -5.671875
     ],
     "tokens": [
       " a",
       " typical",
       " rainy"
     ],
     "top_logprobs": [
       {
         " a": -1.3818359375,
         " my": -2.384765625,
         " the": -1.8720703125
       },
       {
         " big": -1.83984375,
         " day": -2.53125,
         " very": -2.916015625
       },
       {
         " Monday": -2.974609375,
         " day": -1.095703125,
         " work": -2.94140625
       }
     ]
   },
   'text': ' a typical rainy',
   'token_index': 0,
   'prompt_index': 0}])